<a href="https://colab.research.google.com/github/iamnovocaine/inf_search/blob/main/%D0%98%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D0%B8%D1%81%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Тексты9.zip to Тексты9.zip


In [ ]:
import zipfile
with zipfile.ZipFile("Тексты9.zip","r") as zip_ref:
    zip_ref.extractall("Тексты9")
# !unzip Тексты9.zip


In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-12-29 12:57:46--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.242, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-12-29 12:57:47--  http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)... 5.45.220.18, 2a02:6b8:0:2002::19
Connecting to cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)|5.45.220.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [ ]:
from pymystem3 import Mystem
mystem = Mystem(disambiguation=True)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
raw1 = 'Как правильно поливать яблони, чтобы они плодоносили;'
raw1 = 'Каково содержание воды в почве;'
raw1 = 'Каковы потребности овощных культур во влаге;'
raw1 = 'Каким образом в земле сохраняется вода;'
raw1 = 'Какие есть способы посадки плодовых деревьев.'

for txt1 in mystem.lemmatize(raw1):
  if str(txt1).isalpha():
    res1 = mystem.analyze(str(txt1))
    if res1[0]['analysis'] and len(str(txt1)) > 1:
      if res1[0]['analysis'][0]['gr'].startswith("S,") or res1[0]['analysis'][0]['gr'].startswith("A=") or res1[0]['analysis'][0]['gr'].startswith("V") or res1[0]['analysis'][0]['gr'].startswith("A,"):
        print(str(txt1))  

In [ ]:
# четвертое задание
import re
from prettytable import PrettyTable 
import os
import copy
import pandas as pd

terms = {} #словарь слово - количество раз в текстах
rangs = {} #словарь количество раз - ранг
stops = []
documents = {}
filenames = []
vector = list()
termsSorted = {}

requests = [
  ["полив", "яблоня"],
  ["содержание", "вода", "почва"],
  ["потребность", "культура", "влага"],
  ["земля", "вода"],
  ["способ","посадка", "плодовый", "дерево"]
]

metrics = [
    ["17.txt", "18.txt"],
    ["12.txt", "14.txt", "17.txt", "N02.txt", "N03.txt", "N04.txt", "N05.txt", "N06.txt", "N14.txt"],
    ["15.txt", "16.txt", "N06.txt"],
    ["12.txt", "N01.txt", "N02.txt", "N03.txt", "N04.txt", "N05.txt", "N07.txt", "N10.txt", "N12.txt"],
    ["18.txt"]
]

# составляем индекс поисковой системы
for root, dirs, files in os.walk("Тексты9"):  
  for filename in files:
      terms_in_one = {} #словарь слово - количество раз в одном тексте
      raw = []
      # print("открываем  файл " + filename)
      raw = open("Тексты9/" + filename , 'r', encoding='utf8').read()
      
      # записываем в словарь 'terms' термы из документов
      for txt in mystem.lemmatize(raw):
          if str(txt).isalpha():
            res = mystem.analyze(str(txt))
            if res[0]['analysis'] and len(str(txt)) > 1:
              if res[0]['analysis'][0]['gr'].startswith("S,") or res[0]['analysis'][0]['gr'].startswith("A=") or res[0]['analysis'][0]['gr'].startswith("V") or res[0]['analysis'][0]['gr'].startswith("A,"):
                if str(txt) in terms:
                  terms[str(txt)] = terms[str(txt)] + 1
                else:
                  terms[str(txt)] = 1

                if str(txt) in terms_in_one:
                  terms_in_one[str(txt)] = terms_in_one[str(txt)] + 1
                else:
                  terms_in_one[str(txt)] = 1
              else:
                stops.append(str(txt))
      documents[str(filename)] = terms_in_one
      filenames.append(filename)

for word in sorted(terms, key=terms.get, reverse=True)[:500]:
  termsSorted[str(word)] = terms[word]

i = 0
for filename in filenames:    
  vector.append(list())  
  for word in termsSorted:
    if word in documents[filename]:
      vector[i].append(documents[filename][word])
    else:
      vector[i].append(0)
  i += 1

matrix = pd.DataFrame(data=vector, columns=termsSorted.keys())
matrix.index = filenames

matrix['max_tf'] = matrix.max(axis=1).astype(int)
matrix

,почва,вода,растение,влага,почвенный,быть,мочь,полив,корень,влажность,воздух,дерево,количество,вещество,вид,слой,частица,лист,являться,капиллярный,поверхность,большой,участок,температура,условие,часть,содержание,грунт,система,сила,высокий,иметь,земля,время,пора,период,состав,рост,грунтовый,процесс,...,растительность,прорастать,формироваться,учитывать,листва,норма,ряд,конец,погибать,меняться,площадь,увядание,затопление,проницаемость,находить,эфемер,мезофит,дражный,падать,крыло,жидкость,тело,оказываться,превышать,искусственный,плохой,новый,лето,характеристика,проблема,комплекс,позволять,стоить,устраивать,дно,сделать,функция,земляной,заполняться,max_tf
N17.txt,0,7,3,1,0,3,5,0,0,0,0,0,0,0,0,0,0,4,0,0,9,0,0,0,0,0,0,0,0,3,1,1,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,13
N15.txt,11,26,11,5,0,23,8,2,1,0,0,4,2,0,8,4,0,1,3,0,3,3,33,0,3,4,0,5,15,0,5,2,5,1,0,3,0,0,3,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,0,3,1,0,1,1,2,1,2,2,4,2,2,1,2,2,33
N04.txt,48,71,15,4,12,0,3,1,0,13,10,0,12,5,5,0,10,0,4,11,5,6,0,6,2,1,6,0,0,10,2,1,0,1,3,0,6,0,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,71
N10.txt,4,4,20,1,4,1,4,0,7,1,2,1,3,6,3,4,0,2,3,0,0,1,1,0,1,1,1,25,0,0,2,4,7,1,0,1,3,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,2,0,25
16.txt,13,18,15,16,0,2,1,12,7,2,2,0,2,2,1,5,0,7,1,0,1,4,2,3,1,3,3,1,1,0,1,1,0,2,0,4,0,5,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18
N08.txt,54,16,24,7,24,3,5,1,7,0,11,0,2,14,4,11,12,0,13,0,0,3,0,3,2,2,8,0,5,0,5,1,0,0,2,0,3,2,2,13,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,54
N01.txt,10,17,0,4,2,0,2,0,0,0,1,0,2,0,2,2,2,0,2,4,4,1,0,0,0,0,1,4,0,0,2,3,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,17
N03.txt,40,27,18,27,16,1,4,2,2,8,3,0,2,1,0,6,2,0,1,7,2,5,0,1,3,4,6,3,1,7,0,3,2,1,3,1,1,0,10,2,...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40
N05.txt,67,45,7,11,1,2,8,1,2,12,5,0,9,2,3,6,4,0,1,22,3,4,1,1,3,0,5,1,0,5,3,0,0,1,6,1,5,0,6,1,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,67
18.txt,15,13,6,2,0,4,3,29,2,2,3,13,0,0,2,3,0,0,0,0,2,1,3,2,0,1,0,0,4,0,0,0,0,2,1,2,0,4,0,1,...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,2,0,1,0,29


In [ ]:
import math
import copy

Num = len(list(matrix.index))
idfs = {}
for col in matrix.columns:
  idfs[col] = math.log(Num / sum(matrix.apply(lambda item: 1 if item[col] != 0 else 0, axis=1)))
  
matrix_vector_tf_max = copy.deepcopy(matrix)
matrix_vector_tf_max = matrix_vector_tf_max.apply(lambda item: item / item.max_tf, axis=1)
matrix_vector_tf_max = matrix_vector_tf_max.append(pd.Series(idfs.values(), index=idfs.keys(), name="IDFs"))
matrix_vector_tf_max

,почва,вода,растение,влага,почвенный,быть,мочь,полив,корень,влажность,воздух,дерево,количество,вещество,вид,слой,частица,лист,являться,капиллярный,поверхность,большой,участок,температура,условие,часть,содержание,грунт,система,сила,высокий,иметь,земля,время,пора,период,состав,рост,грунтовый,процесс,...,растительность,прорастать,формироваться,учитывать,листва,норма,ряд,конец,погибать,меняться,площадь,увядание,затопление,проницаемость,находить,эфемер,мезофит,дражный,падать,крыло,жидкость,тело,оказываться,превышать,искусственный,плохой,новый,лето,характеристика,проблема,комплекс,позволять,стоить,устраивать,дно,сделать,функция,земляной,заполняться,max_tf
N17.txt,0.000000,0.538462,0.230769,0.076923,0.000000,0.230769,0.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.307692,0.000000,0.000000,0.692308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.230769,0.076923,0.076923,0.076923,0.153846,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.384615,0.538462,0.076923,0.153846,0.076923,0.076923,0.076923,0.076923,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
N15.txt,0.333333,0.787879,0.333333,0.151515,0.000000,0.696970,0.242424,0.060606,0.030303,0.000000,0.000000,0.121212,0.060606,0.000000,0.242424,0.121212,0.000000,0.030303,0.090909,0.000000,0.090909,0.090909,1.000000,0.000000,0.090909,0.121212,0.000000,0.151515,0.454545,0.000000,0.151515,0.060606,0.151515,0.030303,0.000000,0.090909,0.000000,0.000000,0.090909,0.030303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.121212,0.000000,0.030303,0.000000,0.090909,0.030303,0.000000,0.030303,0.030303,0.060606,0.030303,0.060606,0.060606,0.121212,0.060606,0.060606,0.030303,0.060606,0.060606,1.0
N04.txt,0.676056,1.000000,0.211268,0.056338,0.169014,0.000000,0.042254,0.014085,0.000000,0.183099,0.140845,0.000000,0.169014,0.070423,0.070423,0.000000,0.140845,0.000000,0.056338,0.154930,0.070423,0.084507,0.000000,0.084507,0.028169,0.014085,0.084507,0.000000,0.000000,0.140845,0.028169,0.014085,0.000000,0.014085,0.042254,0.000000,0.084507,0.000000,0.028169,0.014085,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
N10.txt,0.160000,0.160000,0.800000,0.040000,0.160000,0.040000,0.160000,0.000000,0.280000,0.040000,0.080000,0.040000,0.120000,0.240000,0.120000,0.160000,0.000000,0.080000,0.120000,0.000000,0.000000,0.040000,0.040000,0.000000,0.040000,0.040000,0.040000,1.000000,0.000000,0.000000,0.080000,0.160000,0.280000,0.040000,0.000000,0.040000,0.120000,0.040000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.040000,0.080000,0.000000,1.0
16.txt,0.722222,1.000000,0.833333,0.888889,0.000000,0.111111,0.055556,0.666667,0.388889,0.111111,0.111111,0.000000,0.111111,0.111111,0.055556,0.277778,0.000000,0.388889,0.055556,0.000000,0.055556,0.222222,0.111111,0.166667,0.055556,0.166667,0.166667,0.055556,0.055556,0.000000,0.055556,0.055556,0.000000,0.111111,0.000000,0.222222,0.000000,0.277778,0.000000,0.055556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [ ]:
matrix_vector_wtj = copy.deepcopy(matrix_vector_tf_max)
for col in matrix_vector_wtj:
  matrix_vector_wtj[col] *= matrix_vector_wtj[col].IDFs
matrix_vector_wtj = matrix_vector_wtj.drop(columns='max_tf')[0:29]
matrix_vector_wtj

,почва,вода,растение,влага,почвенный,быть,мочь,полив,корень,влажность,воздух,дерево,количество,вещество,вид,слой,частица,лист,являться,капиллярный,поверхность,большой,участок,температура,условие,часть,содержание,грунт,система,сила,высокий,иметь,земля,время,пора,период,состав,рост,грунтовый,процесс,...,обусловливать,растительность,прорастать,формироваться,учитывать,листва,норма,ряд,конец,погибать,меняться,площадь,увядание,затопление,проницаемость,находить,эфемер,мезофит,дражный,падать,крыло,жидкость,тело,оказываться,превышать,искусственный,плохой,новый,лето,характеристика,проблема,комплекс,позволять,стоить,устраивать,дно,сделать,функция,земляной,заполняться
N17.txt,0.000000,0.0,0.016491,0.005497,0.000000,0.034251,0.042000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.246876,0.000000,0.000000,0.257236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.270016,0.014557,0.024829,0.045747,0.057164,0.067876,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.872571,1.813159,0.152385,0.304769,0.109337,0.109337,0.135220,0.121195,0.121195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
N15.txt,0.023820,0.0,0.023820,0.010827,0.000000,0.103444,0.026473,0.039954,0.009781,0.000000,0.000000,0.117503,0.011469,0.000000,0.078248,0.033485,0.000000,0.024314,0.025114,0.000000,0.033779,0.013493,0.802346,0.000000,0.021073,0.039124,0.000000,0.090107,0.192208,0.000000,0.028673,0.019562,0.090107,0.011260,0.000000,0.059931,0.000000,0.000000,0.059931,0.014452,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240121,0.000000,0.043072,0.000000,0.159805,0.047744,0.000000,0.047744,0.043072,0.106537,0.047744,0.095487,0.106537,0.274992,0.120061,0.106537,0.047744,0.120061,0.095487
N04.txt,0.048310,0.0,0.015097,0.004026,0.080607,0.000000,0.004614,0.009285,0.000000,0.059099,0.038909,0.000000,0.031985,0.022731,0.022731,0.000000,0.149959,0.000000,0.015564,0.199527,0.026166,0.012543,0.000000,0.035734,0.006530,0.004546,0.050257,0.000000,0.000000,0.164799,0.005331,0.004546,0.000000,0.005233,0.037284,0.000000,0.045134,0.000000,0.018570,0.006717,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
N10.txt,0.011433,0.0,0.057167,0.002858,0.076308,0.005937,0.017472,0.000000,0.090377,0.012911,0.022100,0.038776,0.022709,0.077466,0.038733,0.044201,0.000000,0.064188,0.033150,0.000000,0.000000,0.005937,0.032094,0.000000,0.009272,0.012911,0.023788,0.594707,0.000000,0.000000,0.015139,0.051644,0.166518,0.014863,0.000000,0.026370,0.064090,0.029130,0.000000,0.019077,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070314,0.000000,0.000000,0.000000,0.000000,0.070314,0.000000,0.000000,0.070314,0.000000,0.079240,0.000000,0.063021,0.158480,0.000000
16.txt,0.051609,0.0,0.059549,0.063519,0.000000,0.016491,0.006067,0.439497,0.125523,0.035864,0.030695,0.000000,0.021027,0.035864,0.017932,0.076737,0.000000,0.312024,0.015347,0.000000,0.020642,0.032982,0.089150,0.070476,0.012878,0.053796,0.099118,0.033039,0.023492,0.000000,0.010513,0.017932,0.000000,0.041285,0.000000,0.146499,0.000000,0.202288,0.000000,0.026496,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [ ]:
index = 0
# import warnings
for onerequest in requests:
  print("Запрос: ")
  print(' '.join(onerequest))
  output = {}
  tftq = {}
  idftq = {}
  wtq = {}

  a = 0
  b = 0
  c = 0
  d = 0
  # идем по словам в запросе
  for one_term_in_request in onerequest:
    tftq[one_term_in_request] = onerequest.count(one_term_in_request)
    if one_term_in_request in idfs: 
      idftq[one_term_in_request] = idfs[one_term_in_request]
    else: 
      idftq[one_term_in_request] = 0
  for one_term_in_request in onerequest:
    wtq[one_term_in_request] = (0.5 + (0.5 * tftq[one_term_in_request]) / max(tftq.values())) * idftq[one_term_in_request]
    if one_term_in_request not in matrix_vector_wtj:
      matrix_vector_wtj[one_term_in_request] = 0
  print(wtq)
  i = 0


  while i < Num:  
    sim = np.dot(list(matrix_vector_wtj[onerequest].iloc[i]), list(wtq.values())) / (np.linalg.norm(list(matrix_vector_wtj[onerequest].iloc[i])) \
      * np.linalg.norm(list(wtq.values())))
    
    if math.isnan(sim):
      sim = 0
    if sim > 0: # можем менять отсечку релевантности
      output[matrix_vector_wtj.index[i]] = sim
    i += 1
    output_sorted = {k: v for k, v in sorted(output.items(), key=lambda item: -item[1])}

    i += 1

  for doc in output_sorted.keys():
    print(doc, output_sorted[doc])
    if doc in metrics[index]:
      a = a + 1
    else:
      # print("b нашелся нерелевантный " + doc)
      b = b + 1
  c = len(metrics[index]) - a
  d = len(filenames) - len(metrics[index]) - b

  print("d = " + str(d) + ", a = " + str(a) + ", b = " + str(b) + ", c = " + str(c))

  recall = a / (a + c)
  precision = a / (a + b)
  fallout = b / (b + d)
  accuracy = (a + d) / (a + b + c + d)

  if recall == 0 or precision == 0:
    fvalue = 0
  else:
    fvalue = 2 / (1 / precision + 1 / recall)
  print("recall= " + str(recall) + ", precision = " + str(precision) + ", fallout = " + str(fallout))
  print("accuracy= " + str(accuracy) + ", fvalue = " + str(fvalue) + "\n\n")
  print("----\n")

  index = index + 1


Запрос: 
полив яблоня
{'полив': 0.659245628884264, 'яблоня': 0.0}
N04.txt 1.0
16.txt 1.0
N05.txt 1.0
N18.txt 1.0
15.txt 1.0
17.txt 1.0
12.txt 1.0
d = 21, a = 1, b = 6, c = 1
recall= 0.5, precision = 0.14285714285714285, fallout = 0.2222222222222222
accuracy= 0.7586206896551724, fvalue = 0.2222222222222222


----

Запрос: 
содержание вода почва
{'содержание': 0.5947071077466928, 'вода': 0.0, 'почва': 0.07145896398214505}
11.txt 0.9998042693007199
16.txt 0.9357297409311813
N04.txt 0.798458249864272
N01.txt 0.7268162866164439
N05.txt 0.6251738639994153
12.txt 0.5860085502935362
13.txt 0.5161245757678211
17.txt 0.4915845277571879
14.txt 0.3803660985523272
N18.txt 0.11930010636037537
N13.txt 0.11930010636037537
15.txt 0.11930010636037536
N02.txt 0.11930010636037536
d = 13, a = 6, b = 7, c = 3
recall= 0.6666666666666666, precision = 0.46153846153846156, fallout = 0.35
accuracy= 0.6551724137931034, fvalue = 0.5454545454545455


----

Запрос: 
потребность культура влага
{'потребность': 1.57553

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# третье задание
import re
from prettytable import PrettyTable 
import os
import copy

terms = {} #словарь слово - количество раз в текстах
rangs = {} #словарь количество раз - ранг
stops = []
documents = {}
filenames = []
th = []
column_names = []
fff = {}



requests = [
  ["поливать", "яблоня"],
  ["содержание", "вода", "почва"],
  ["потребность", "культура", "влага"],
  ["земля", "вода"],
  ["способ","посадка", "плодовый", "дерево"]
]

metrics = [
    ["17.txt", "18.txt"],
    ["12.txt", "14.txt", "17.txt", "N02.txt", "N03.txt", "N04.txt", "N05.txt", "N06.txt", "N14.txt"],
    ["15.txt", "16.txt", "N06.txt"],
    ["12.txt", "N01.txt", "N02.txt", "N03.txt", "N04.txt", "N05.txt", "N07.txt", "N10.txt", "N12.txt"],
    ["18.txt"]
]
# составляем индекс поисковой системы
for root, dirs, files in os.walk("Тексты9"):  
  for filename in files:
      terms_in_one = {} #словарь слово - количество раз в одном тексте
      raw = []
      # print("открываем  файл " + filename)
      raw = open("Тексты9/" + filename , 'r', encoding='utf8').read()
      
      # записываем в словарь 'terms' термы из документов
      for txt in mystem.lemmatize(raw):
          if str(txt).isalpha():
            res = mystem.analyze(str(txt))
            if res[0]['analysis'] and len(str(txt)) > 1:
              if res[0]['analysis'][0]['gr'].startswith("S,") or res[0]['analysis'][0]['gr'].startswith("A=") or res[0]['analysis'][0]['gr'].startswith("V") or res[0]['analysis'][0]['gr'].startswith("A,"):
                if str(txt) in terms:
                  terms[str(txt)] = terms[str(txt)] + 1
                else:
                  terms[str(txt)] = 1

                if str(txt) in terms_in_one:
                  terms_in_one[str(txt)] = terms_in_one[str(txt)] + 1
                else:
                  terms_in_one[str(txt)] = 1
              else:
                stops.append(str(txt))
      documents[str(filename)] = terms_in_one
      filenames.append(filename)

# считаем ранг частоты термов и записываем в словарь 'rangs'
for word in sorted(terms, key=terms.get, reverse=True)[:600]:
  # print(word, terms[word], fff[word])
  th.append(word)

# print(th)
index = 0

for onerequest in requests:
  print("Запрос: ")
  print(' '.join(onerequest))

  # print(metrics[index])
  files = {}
  count_mas = len(onerequest)

  a = 0
  b = 0
  c = 0
  d = 0
  # идем по словам в запросе
  for one_term_in_request in onerequest:    
    # print("ищем слово " + one_term_in_request)
    # if one_term_in_request in th:
      for doc in documents:
        if one_term_in_request in documents[doc]:
          if str(doc) in files:
            files[str(doc)] = files[str(doc)] + 1
          else:
            files[str(doc)] = 1
  # print(files)
  
  for file in files:
    if files[file] == count_mas:
      print("нашелся в файле " + file)
      if file in metrics[index]:
        a = a + 1
      else:
        # print("b нашелся нерелевантный " + file)
        b = b + 1
  for file in metrics[index]:
    # print(files[str(file)])
    if file in files and files[file] == count_mas: 
      nn = 2
      # print("есть " + file)
    else:
      c = c + 1

  print("------\n\n")

  # if a == 0:
  #   a = 1
  d = len(filenames) - len(metrics[index])
  print("d = " + str(d) + ", a = " + str(a) + ", b = " + str(b) + ", c = " + str(c))
  recall = a / (a + c)
  precision = a / (a + b)
  fallout = b / (b + d)
  accuracy = (a + d) / (a + b + c + d)

  if recall == 0 or precision == 0:
    fvalue = 0
  else:
    fvalue = 2 / (1 / precision + 1 / recall)
  print("recall= " + str(recall) + ", precision = " + str(precision) + ", fallout = " + str(fallout))
  print("accuracy= " + str(accuracy) + ", fvalue = " + str(fvalue) + "\n\n")

  index = index + 1

# здесь можно вывести полный частотный словарь всех файлов
# for word in sorted(terms, key=terms.get, reverse=True):
#   print(word, terms[word])


  


Запрос: 
поливать яблоня
нашелся в файле 17.txt
нашелся в файле 18.txt
------


d = 27, a = 2, b = 0, c = 0
recall= 1.0, precision = 1.0, fallout = 0.0
accuracy= 1.0, fvalue = 1.0


Запрос: 
содержание вода почва
нашелся в файле N14.txt
нашелся в файле 16.txt
нашелся в файле N12.txt
нашелся в файле 11.txt
нашелся в файле N05.txt
нашелся в файле N07.txt
нашелся в файле N04.txt
нашелся в файле 14.txt
нашелся в файле N16.txt
нашелся в файле N03.txt
нашелся в файле 17.txt
нашелся в файле 13.txt
нашелся в файле N08.txt
нашелся в файле N01.txt
нашелся в файле 12.txt
нашелся в файле N10.txt
------


d = 20, a = 7, b = 9, c = 2
recall= 0.7777777777777778, precision = 0.4375, fallout = 0.3103448275862069
accuracy= 0.7105263157894737, fvalue = 0.56


Запрос: 
потребность культура влага
нашелся в файле 11.txt
нашелся в файле N15.txt
нашелся в файле N13.txt
нашелся в файле 18.txt
------


d = 26, a = 0, b = 4, c = 3
recall= 0.0, precision = 0.0, fallout = 0.13333333333333333
accuracy= 0.7878787878

In [ ]:
# второе задание
import re
from prettytable import PrettyTable  # Импортируем установленный модуль.
import os

terms = {} #словарь слово - количество раз в текстах
rangs = {} #словарь количество раз - ранг
stops = []
documents = []
filenames = []
th = []
column_names = []

for root, dirs, files in os.walk("Тексты9"):  
  for filename in files:
      terms_in_one = {} #словарь слово - количество раз в одном тексте
      raw = []
      # print("открываем  файл " + filename)
      raw = open(filename , 'r', encoding='utf8').read()
      
      # записываем в словарь 'terms' термы из документов
      for txt in mystem.lemmatize(raw):
          if str(txt).isalpha():
            res = mystem.analyze(str(txt))
            if res[0]['analysis'] and len(str(txt)) > 1:
              if res[0]['analysis'][0]['gr'].startswith("S,") or res[0]['analysis'][0]['gr'].startswith("A=") or res[0]['analysis'][0]['gr'].startswith("V") or res[0]['analysis'][0]['gr'].startswith("A,"):
                if str(txt) in terms:
                  terms[str(txt)] = terms[str(txt)] + 1
                else:
                  terms[str(txt)] = 1
                # -------------
                if str(txt) in terms_in_one:
                  terms_in_one[str(txt)] = terms_in_one[str(txt)] + 1
                else:
                  terms_in_one[str(txt)] = 1
              else:
                stops.append(str(txt))
      documents.append(terms_in_one)
      filenames.append(filename)

# здесь можно вывести полный частотный словарь всех файлов
# for word in sorted(terms, key=terms.get, reverse=True):
#   print(word, terms[word])

# здесь можно вывести список стоп-слов
# print(stops)

# считаем ранг частоты термов и записываем в словарь 'rangs'

for word in sorted(terms, key=terms.get, reverse=True)[:500]:
  # print(word, terms[word])
  th.append(word)

table = PrettyTable(th)  # Определяем таблицу.

u = 0
for Value in documents:
  empty = []
  column_names.append(filenames[u])
  for key in th:
    if key in Value:
      empty.append(Value[key])
    else:
      empty.append(0)
  table.add_row(empty)
  u = u + 1

# добавим столбец с названиями файлов
fieldname = 'Имя файла'
table._field_names.insert(0, fieldname) 
table._align[fieldname] = 'c' 
table._valign[fieldname] = 't' 
pos = 0;
for one in column_names: 
  table._rows[pos].insert(0, one) 
  pos = pos + 1

print(table)
  


+-----------+-------+------+----------+-------+-----------+------+------+-------+--------+-----------+--------+--------+------------+----------+------+-----+---------+------+-------------+----------+---------+-------------+---------+-------------+-------+---------+------------+-------+---------+------+---------+-------+-------+------+-------+--------+--------+------+-----------+----------+---------+--------+----+----------+-----------+-------+--------------+----------+------------+-------------+----------+--------------+------------------+-------------+---------+-------+-------+----------+-----------+---------+-------+-------+------------+-------------+-----------+-----------+-----------+--------+-----------+------+----------+--------------+-------------+--------+----------------+------------+-------+----------+---------+-------------+--------+---------+---------+-----------+-------------+------------+-----------+-----+----------+---------+-----------+------+---------+----------+------

In [ ]:
# первое задание
i = 11
#http://www.itstan.ru/poiskovye-sistemy/zakony-zipfa.html
import re
length_terms = 0
while i < 39:
  raw = []

  terms = {} #словарь слово - количество раз в тексте
  rangs = {} #словарь количество раз - ранг
  terms_rangs = {} #словарь слово - ранг
  Zipf = []
  frequency = []

  j = "Тексты9/" + str(i) + ".txt"
  print("открываем  файл " + j)
  raw = open(j , 'r', encoding='utf8').read()

  # записываем в словарь 'terms' термы из документа
  for txt in mystem.lemmatize(raw):
      if str(txt).isalpha():
        if str(txt) in terms:
          terms[str(txt)] = terms[str(txt)] + 1
        else:
          terms[str(txt)] = 1

  # считаем ранг частоты термов и записываем в словарь 'rangs'
  num = 0;
  for word in sorted(terms, key=terms.get, reverse=True):
    #print(word, terms[word])
    if terms[word] in rangs:
      rangs[terms[word]] = num
    else:
      num = num + 1
      rangs[terms[word]] = num

  #print(rangs)
  for word in rangs:
    length_terms = length_terms + word
  print("Число термов в документе: " + str(length_terms))
  
  # считаем постоянную Зипфа для документа
  for word in rangs:
    frequency.append(word / length_terms)
    Zipf.append(word / length_terms * rangs[word])  

  print(Zipf)
  ZipfConstant = sum(Zipf) / len(Zipf)
  print("Постоянная Зипфа: " + str(ZipfConstant))

  fig, ax = plt.subplots()
  ax.set_xlim([0, max(rangs.values())])
  #ax.set_ylim([0, max(rangs.keys())])
  ax.set_ylim([0, max(frequency)])

  ax.set_xlabel('ранг')
  ax.set_ylabel('частота')
  
  ax.plot(np.array(list(rangs.values())).astype(float), np.array(list(frequency)).astype(float))
  plt.show()

  i = i + 1
